# Python - Numba

---

A **Just-In-Time (JIT) compiler** is a system that compiles code during the execution of a program rather than ahead of time before running.

It translates code (often bytecode or intermediate code) into native machine code on the fly, right before it needs to execute.

In [1]:
import numba as nb
import numpy as np
from scipy.stats import norm

from theoria.validor import TestCase, Validor

if __name__ == "__main__":
    np.random.seed(42)

## Example: Vanilla Call Option Pricing

In [2]:
def price_numpy(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    n_simulations: int,
) -> float:
    discount = np.exp(-r * T)
    drift = (r - 0.5 * sigma * sigma) * T
    diffusion = sigma * np.sqrt(T)

    W = np.random.standard_normal(n_simulations)
    ST = S0 * np.exp(drift + diffusion * W)
    return discount * np.mean(np.maximum(ST - K, 0))


@nb.njit(parallel=True, fastmath=True)
def price_numba(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    n_simulations: int,
) -> float:
    discount = np.exp(-r * T)
    drift = (r - 0.5 * sigma * sigma) * T
    diffusion = sigma * np.sqrt(T)

    payoffs = 0.0

    for _ in nb.prange(n_simulations):
        W = np.random.normal()
        ST = S0 * np.exp(drift + diffusion * W)
        payoffs += max(ST - K, 0)

    return discount * payoffs / n_simulations


def price_analytical(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
) -> float:
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

In [3]:
%%timeit -n 10 -r 1

# Vectorized NumPy pricing
numpy_price = price_numpy(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    n_simulations=10_000_000,
)

314 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [4]:
%%timeit -n 10 -r 1

# Numba-optimized pricing
numba_price = price_numba(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    n_simulations=10_000_000,
)

215 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [5]:
%%timeit -n 10 -r 1

# Analytical pricing
analytical_price = price_analytical(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
)

293 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [6]:
num_iterations = 100_000_000


def comparison(
    actual_price: float,
    expected_price: float,
    rel_tol: float = 1e-3,
) -> bool:
    return abs((actual_price - expected_price) / expected_price) < rel_tol


test_cases = [
    TestCase(
        input_data={
            "S0": 100,
            "K": 100,
            "T": 1,
            "r": 0.05,
            "sigma": 0.2,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=100,
            K=100,
            T=1,
            r=0.05,
            sigma=0.2,
        ),
        description="MC price should approximate analytical price",
    ),
    TestCase(
        input_data={
            "S0": 150,
            "K": 100,
            "T": 0.5,
            "r": 0.03,
            "sigma": 0.25,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=150,
            K=100,
            T=0.5,
            r=0.03,
            sigma=0.25,
        ),
        description="MC price should approximate analytical price for ITM option",
    ),
    TestCase(
        input_data={
            "S0": 80,
            "K": 100,
            "T": 2,
            "r": 0.04,
            "sigma": 0.3,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=80,
            K=100,
            T=2,
            r=0.04,
            sigma=0.3,
        ),
        description="MC price should approximate analytical price for OTM option",
    ),
]

Validor(price_numpy).add_cases(test_cases).run(comparison)
Validor(price_numba).add_cases(test_cases).run(comparison)

[2026-01-09 20:39:07,513] [INFO] All 3 tests passed for price_numpy.


[2026-01-09 20:39:10,520] [INFO] All 3 tests passed for price_numba.
